In [1]:
from itertools import permutations
from tqdm.autonotebook import tqdm
import math

/Users/maximefournes/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [42]:
"""
 AB
H  C
G  D
 FE
"""
VERTICES = "abcdefgh"
ROTATIONS = "ac,bd,ce,df,eg,fh,ga,hb"
N_ROTATIONS = 4

In [7]:
"""
 ABC
L   D
K   E
J   F
 IHG

"""

VERTICES = "abcdefghijkl"
ROTATIONS = "ad,be,cf,dg,eh,fi,gj,hk,il,ja,kb,lc"
N_ROTATIONS = 4

In [44]:
"""
    .
  d/ \e
 c/   \f
 |     |
b|     |g
a|     |h
 |     |
 l\   /i
  k\ /j
    .
"""
VERTICES = "abcdefghijkl"
ROTATIONS = "ac,bd,ce,df,eg,fh,gi,hj,ik,jl,ka,lb"
N_ROTATIONS = 6

In [54]:
"""
 B C
A   D
 F E
"""

VERTICES = "abcdef"
ROTATIONS = "ab,bc,de,de,ef,fa"
N_ROTATIONS = 6

In [56]:
"""
   .
 B. .C
A.   .D
.......
  F E
"""
VERTICES = "abcdef"
ROTATIONS = "ab,bc,de,de,ef,fa"
N_ROTATIONS = 3

In [60]:
VERTICES = "abcdefghijklmnopqr"
ROTATIONS = "ad,be,cf,dg,eh,fi,gj,hk,il,jm,kn,lo,mp,nq,or"
N_ROTATIONS = 6

In [ ]:
def gen_permuts(vertices, verbose=True):
    if len(vertices) == 0:
        return []
    if len(vertices) == 2:
        return [[(vertices[0], vertices[1])]]
    permuts = []
    v0, remaining = vertices[0], vertices[1:]
    for i, v1 in tqdm(enumerate(remaining), disable=not verbose):
        permuts.extend([[(v0, v1)] + perm for perm in gen_permuts(remaining[:i]+remaining[i+1:], verbose=False)])
    return permuts
            

rotations = dict(ROTATIONS.split(","))

def rotate_vertex(v, rotations=rotations):
    return rotations.get(v)


class Tile:
    def __init__(self, edge_map):
        self.edge_map = edge_map
    
    def rotate(self):
        new_edges = {}
        for v in VERTICES:
            rot_v = rotate_vertex(v)
            con_v = self.edge_map.get(v)
            unrot_v = rotate_vertex(con_v)
            new_edges[rot_v] = unrot_v
        return Tile(new_edges)

    def iter_rotations(self):
        tile = self.rotate()
        for i in range(N_ROTATIONS):
            tile = tile.rotate()
            yield tile
    
    def hash_for_fixed_rot(self):
        return hash(str(self.edge_map))
    
    def is_valid(self):
        for v in VERTICES:
            connected_to_v = self.edge_map.get(v)
            connected_to_connected_to_v = self.edge_map.get(connected_to_v)
            if v != connected_to_connected_to_v or connected_to_v == v:
                return False
        return True
    
    def __repr__(self):
        mems = [f"{k}->{v}" for k, v in self.edge_map.items()]
        return f"Tile({mems})"
    
    def __hash__(self):
        hashes = [tile.hash_for_fixed_rot() for tile in self.iter_rotations()]
        return min(hashes)

    
all_tiles = {}

all_permuts = gen_permuts(VERTICES)
for perm in tqdm(all_permuts):
    emap = dict(perm+[(v1, v0) for v0, v1 in perm])
    tile = Tile(emap)
    if tile.is_valid():
        all_tiles[hash(tile)] = tile
        
len(all_tiles)